##### ***Import libraries***

In [1]:
import pandas as pd
import numpy as np 
from tqdm import tqdm
import flatten_json
import pymongo
import math

##### ***Data loading***

In [2]:
#Load reviews dataset
reviews = pd.read_json("yelp_academic_dataset_review.json", lines = True, convert_dates = ["date"])

#Load businesses dataset
businesses = pd.read_json("yelp_academic_dataset_business.json", lines = True)

#Load users dataset
users = pd.read_json("yelp_academic_dataset_user.json", lines = True, convert_dates = ["yelping_since"])

#Load checkins dataset
checkins = pd.read_json("yelp_academic_dataset_checkin.json", lines = True)

##### ***Data inspection***

In [3]:
#Info of reviews
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 480.0+ MB


In [4]:
#Some elements of reviews
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [5]:
#Info of businesses
businesses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB


In [6]:
#Some elements of businesses
businesses.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [7]:
#Info of users
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 22 columns):
 #   Column              Dtype         
---  ------              -----         
 0   user_id             object        
 1   name                object        
 2   review_count        int64         
 3   yelping_since       datetime64[ns]
 4   useful              int64         
 5   funny               int64         
 6   cool                int64         
 7   elite               object        
 8   friends             object        
 9   fans                int64         
 10  average_stars       float64       
 11  compliment_hot      int64         
 12  compliment_more     int64         
 13  compliment_profile  int64         
 14  compliment_cute     int64         
 15  compliment_list     int64         
 16  compliment_note     int64         
 17  compliment_plain    int64         
 18  compliment_cool     int64         
 19  compliment_funny    int64         
 20  co

In [8]:
#Some elements of users
users.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [9]:
#Info of checkins
checkins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  131930 non-null  object
 1   date         131930 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


In [10]:
#Some elements of checkins
checkins.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


##### ***Data cleaning***

In [11]:
#Check for null values in reviews DF
reviews.isnull().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

In [12]:
#Check for null values in businesses DF
businesses.isnull().sum()

business_id         0
name                0
address             0
city                0
state               0
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      13744
categories        103
hours           23223
dtype: int64

In [13]:
#Some null values have been found. First, we compute their perecentage #Ricordarsi di trasformare in lista le categories
num_null_attributes = len(businesses[businesses["attributes"].isna()])
num_null_categories = len(businesses[businesses["categories"].isna()])
num_null_hours = len(businesses[businesses["hours"].isna()])

print(f"""Percentage null "atributes": {num_null_attributes / len(businesses)}""")
print(f"""Percentage null "categories": {num_null_categories / len(businesses)}""")
print(f"""Percentage null "hours": {num_null_hours / len(businesses)}""")

Percentage null "atributes": 0.0914158008859564
Percentage null "categories": 0.0006850864007023798
Percentage null "hours": 0.15446370372341134


In [14]:
#Inspect the meaning of the three columns by printing a non-null value
print("Attributes:")
print(businesses[~businesses["attributes"].isna()]["attributes"].iloc[[0, 2, 3, 4]])
print("\n")

print("Categories:")
print(businesses[~businesses["categories"].isna()]["categories"].iloc[[0, 2, 3, 4]])
print("\n")

print("Hours:")
print(businesses[~businesses["hours"].isna()]["hours"].iloc[[0, 2, 3, 4]])

#No rows will be dropped; MongoDB will handle them.

Attributes:
0                        {'ByAppointmentOnly': 'True'}
2    {'BikeParking': 'True', 'BusinessAcceptsCredit...
3    {'RestaurantsDelivery': 'False', 'OutdoorSeati...
4    {'BusinessAcceptsCreditCards': 'True', 'Wheelc...
Name: attributes, dtype: object


Categories:
0    Doctors, Traditional Chinese Medicine, Naturop...
2    Department Stores, Shopping, Fashion, Home & G...
3    Restaurants, Food, Bubble Tea, Coffee & Tea, B...
4                            Brewpubs, Breweries, Food
Name: categories, dtype: object


Hours:
1    {'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...
3    {'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...
4    {'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...
5    {'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...
Name: hours, dtype: object


In [15]:
#Check for null values in users DF
users.isnull().sum()

user_id               0
name                  0
review_count          0
yelping_since         0
useful                0
funny                 0
cool                  0
elite                 0
friends               0
fans                  0
average_stars         0
compliment_hot        0
compliment_more       0
compliment_profile    0
compliment_cute       0
compliment_list       0
compliment_note       0
compliment_plain      0
compliment_cool       0
compliment_funny      0
compliment_writer     0
compliment_photos     0
dtype: int64

In [16]:
#Check for null values in reviews DF
reviews.isnull().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

#### ***Data normalization***

In [17]:
#We drop column "is_open" because it's not meaningful for the analysis we'd like to perform
businesses.drop(["is_open"], axis = 1, inplace = True)

#Flatten "categories" from nested JSON 
businesses["attributes"] = businesses["attributes"].apply(lambda x : flatten_json.flatten(x) if pd.notna(x) else np.NaN)

#Convert "categories" to array
businesses["categories"] = businesses["categories"].apply(lambda x : x.split(", ") if pd.notna(x) else np.NAN)

In [18]:
#Final result
businesses

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,{'ByAppointmentOnly': 'True'},"[Doctors, Traditional Chinese Medicine, Naturo...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,{'BusinessAcceptsCreditCards': 'True'},"[Shipping Centers, Local Services, Notaries, M...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","[Department Stores, Shopping, Fashion, Home & ...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","[Restaurants, Food, Bubble Tea, Coffee & Tea, ...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","[Brewpubs, Breweries, Food]","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","[Nail Salons, Beauty & Spas]","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","[Pets, Nurseries & Gardening, Pet Stores, Hobb...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","[Shopping, Jewelry, Piercing, Toy Stores, Beau...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,"{'BusinessParking': '{'garage': False, 'street...","[Fitness/Exercise Equipment, Eyewear & Opticia...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


In [19]:
#Investigate column "elite"
print(f"Before: {pd.Series(users["elite"]).str.split(",").explode().unique()}")

#There is a "20" (probably 2020), so we transform it into 2020
def clean_years(x):
    if x != "":
        years = x.split(',')
        years = ['2020' if y.strip()=='20' else y.strip() for y in years]
        return ','.join(years)
    else:
        return np.NaN

users["elite"] = users["elite"].apply(clean_years)

#See the result
print(f"After: {pd.Series(users["elite"]).str.split(",").explode().unique()}")

Before: ['2007' '2009' '2010' '2011' '2012' '2013' '2014' '2015' '2016' '2017'
 '2018' '2019' '20' '2021' '' '2006' '2008']
After: ['2007' '2009' '2010' '2011' '2012' '2013' '2014' '2015' '2016' '2017'
 '2018' '2019' '2020' '2021' nan '2006' '2008']


In [20]:
#Split "elite" into a list
users['elite'] = users['elite'].apply(lambda x: x.split(',') if pd.notna(x) else np.NaN)

In [21]:
#Some "friens" of users DF are "None"; we need to investigate them
none_f = users[users["friends"] == "None"]

for col in ['average_stars', 'compliment_hot',
            'compliment_more', 'compliment_profile', 'compliment_cute',
            'compliment_list', 'compliment_note', 'compliment_plain',
            'compliment_cool', 'compliment_funny', 'compliment_writer',
            'compliment_photos', 'fans']:
    print(f"Column {col}: {none_f[col].unique()[:10]}")

print(f"Percentage: {len(none_f) / len(users)}")
#We have about 44.2% of rows wth "None" in friends: it is better to not drop them.

Column average_stars: [4.   2.83 3.28 4.67 3.22 4.22 3.6  5.   3.81 4.5 ]
Column compliment_hot: [ 0  1  2  3  5 56  4  6  8  7]
Column compliment_more: [0 1 2 3 6 4 8 9 5 7]
Column compliment_profile: [ 0  1  2  3  4 40]
Column compliment_cute: [  0   1   2   3   5  48 116]
Column compliment_list: [ 0  1  2 32]
Column compliment_note: [0 1 2 7 3 4 8 5 6 9]
Column compliment_plain: [ 0  1  2  5  3  6  4 10 13 14]
Column compliment_cool: [ 0  2  1  3  7  4  8  5 16 10]
Column compliment_funny: [ 0  2  1  3  7  4  8  5 16 10]
Column compliment_writer: [ 0  2  4  1  3  6  7  5 17 34]
Column compliment_photos: [  0   1   2   3 252   5   9   4   6   8]
Column fans: [ 0  2  1  3  4 17 19  8  5 11]
Percentage: 0.4419499601840538


In [22]:
#Split "friends" column into lists
users['friends'] = users['friends'].apply(lambda x: x.split(', ') if x != "None" else np.NaN)

In [23]:
#Check that all the friends of a user have their respective user document
friends_of_users = set(users['friends'].explode().dropna().unique())
tot_users = set(users["user_id"])

friends_of_users.issubset(tot_users)

False

In [24]:
#"friends_of_user" is not a subset of "tot_users", so we need to filter out elements
def filter_list(x):
    if x is not np.NaN:  
        return [friend for friend in x if friend not in diff]
    return x

diff = friends_of_users.difference(tot_users)

users['friends'] = users['friends'].apply(filter_list)

#Check if all has gone right
friends_of_users = set(users['friends'].explode().dropna().unique())

friends_of_users.issubset(tot_users)

True

In [25]:
#Final result
users

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,[2007],"[NSCy54eWehBJyZdG2iE84w, 6EDA76lfkaTyOeuxkg0Fm...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"[2009, 2010, 2011, 2012, 2013, 2014, 2015, 201...","[ueRPE0CX75ePGMqOFVj6IQ, E_GAXhVA1_lVC2aFpMQEl...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"[2009, 2010, 2011, 2012, 2013]","[pypZb3V5TXHOnlTj-qLSrw, irkRHMqg9oSt7lv3OSiNk...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"[2009, 2010, 2011]","[enx1vVPnfdNUdPho6PH_wg, 1OocYCAZixwbAXueW75FM...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,NaN,"[tjm3b3Ts8msuGiYRybHx0Q, Z1MFQFgn-O9ADIOr6g-wE...",1,...,1,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987892,fB3jbHi3m0L2KgGOxBv6uw,Jerrold,23,2015-01-06 00:31:31,7,0,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
1987893,68czcr4BxJyMQ9cJBm6C7Q,Jane,1,2016-06-14 07:20:52,0,0,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
1987894,1x3KMskYxOuJCjRz70xOqQ,Shomari,4,2017-02-04 15:31:58,1,1,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
1987895,ulfGl4tdbrH05xKzh5lnog,Susanne,2,2011-01-14 00:29:08,0,0,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
#Check that all business IDs represent real objects
tot_business_in_reviews_ids = set(reviews["business_id"]) 
tot_business_ids = set(businesses["business_id"]) 

tot_business_in_reviews_ids.issubset(tot_business_ids)

True

In [27]:
#Check that all users IDs represent real objects
tot_users_in_reviews_ids = set(reviews["user_id"]) 
tot_users_ids = set(users["user_id"]) 

tot_users_in_reviews_ids.issubset(tot_users_ids)

False

In [28]:
#There are some users that have written reviews but they are not in "users" collection, so we drop them (in this case, given that "diff" is small, we prefer to not altering number of reviews, stars for businesses...)
diff = tot_users_in_reviews_ids.difference(tot_users_ids)
diff = list(diff)

reviews = reviews[~reviews["user_id"].isin(diff)]

#Check if the result is correct
tot_users_in_reviews_ids = set(reviews["user_id"]) 

tot_users_in_reviews_ids.issubset(tot_users_ids)

True

In [29]:
#Final result
reviews

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,"This spot offers a great, affordable east week...",2021-03-31 16:55:10
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27


In [30]:
#Check that all business IDs represent real objects
tot_business_in_checkins_ids = set(checkins["business_id"]) 
tot_business_ids = set(businesses["business_id"]) 

tot_business_in_checkins_ids.issubset(tot_business_ids)

True

In [38]:
#Split "date" into a list
checkins["date"] = checkins["date"].str.split(", ")
checkins["date"] = checkins["date"].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S'))

#Then, convert in a MongoDB-compatible format
checkins["date"] = checkins["date"].apply(lambda x: [date.to_pydatetime() for date in x])

In [39]:
#Final result
checkins

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"[2020-03-13 21:10:56, 2020-06-02 22:18:06, 202..."
1,--0iUa4sNDFiZFrAdIWhZQ,"[2010-09-13 21:43:09, 2011-05-04 23:08:15, 201..."
2,--30_8IhuyMHbSOcNWd6DQ,"[2013-06-14 23:29:17, 2014-08-13 23:20:22]"
3,--7PUidqRWpRSpXebiyxTg,"[2011-02-15 17:12:00, 2011-07-28 02:46:10, 201..."
4,--7jw19RH9JKXgFohspgQw,"[2014-04-21 20:42:11, 2014-04-28 21:04:46, 201..."
...,...,...
131925,zznJox6-nmXlGYNWgTDwQQ,"[2013-03-23 16:22:47, 2013-04-07 02:03:12, 201..."
131926,zznZqH9CiAznbkV6fXyHWA,[2021-06-12 01:16:12]
131927,zzu6_r3DxBJuXcjnOYVdTw,"[2011-05-24 01:35:13, 2012-01-01 23:44:33, 201..."
131928,zzw66H6hVjXQEt0Js3Mo4A,"[2016-12-03 23:33:26, 2018-12-02 19:08:45]"


##### ***Load collections in MongoDB***

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["yelp"]

In [41]:
batch_size = 1000

def load_dataset(db, df, collection_name):
    #Calculate total number of batches
    total_batches = math.ceil(len(df) / batch_size)

    # Process each batch
    for i in tqdm(range(total_batches)):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(df))
        
        #Get batch and convert to dictionary
        batch_df = df.iloc[start_idx : end_idx]
        
        #Convert to dictionary and handle any non-MongoDB compatible data types
        records = batch_df.to_dict("records")
        
        try:
            #Insert batch if non empty
            if records: 
                db[collection_name].insert_many(records)
        except Exception as e:
            print(f"Error inserting batch {i}: {e}")

In [42]:
to_be_inserted = {
    "reviews" : reviews,
    "businesses" : businesses,
    "users" : users,
    "checkins" : checkins
}

for (c_name, df) in to_be_inserted.items():
    print(f"Loading dataset {c_name}:")
    load_dataset(db, df, c_name)
    print("\n")

Loading dataset reviews:


100%|██████████| 6991/6991 [02:50<00:00, 41.02it/s]




Loading dataset businesses:


100%|██████████| 151/151 [00:04<00:00, 36.76it/s]




Loading dataset users:


100%|██████████| 1988/1988 [00:55<00:00, 35.52it/s]




Loading dataset checkins:


100%|██████████| 132/132 [00:05<00:00, 24.47it/s]

##### ***Create indeces on collections***

In [6]:
#For "businesses", we need only "business_id" as index
db["businesses"].create_index(["business_id"], unique = True)

#For "users", we need only "user_id" as index
db["users"].create_index(["user_id"], unique = True)

#For "checkins", we need only "business_id" as index
db["checkins"].create_index(["business_id"], unique = True)

#For "reviews", we need "business_id", "user_id" and "review_id" as index
db["reviews"].create_index(["business_id"])
db["reviews"].create_index(["user_id"])
db["reviews"].create_index(["review_id"], unique = True)

'review_id_1'

In [47]:
#Check the result
print(f"""Indeces of "businesses" collection: {db["businesses"].index_information()}""")
print(f"""Indeces of "users" collection: {db["users"].index_information()}""")
print(f"""Indeces of "checkins" collection: {db["checkins"].index_information()}""")
print(f"""Indeces of "reviews" collection: {db["reviews"].index_information()}""")

Indeces of "businesses" collection: {'_id_': {'v': 2, 'key': [('_id', 1)]}, 'business_id_1': {'v': 2, 'key': [('business_id', 1)]}}
Indeces of "users" collection: {'_id_': {'v': 2, 'key': [('_id', 1)]}, 'user_id_1': {'v': 2, 'key': [('user_id', 1)]}}
Indeces of "checkins" collection: {'_id_': {'v': 2, 'key': [('_id', 1)]}, 'business_id_1': {'v': 2, 'key': [('business_id', 1)]}}
Indeces of "reviews" collection: {'_id_': {'v': 2, 'key': [('_id', 1)]}, 'business_id_1': {'v': 2, 'key': [('business_id', 1)]}, 'user_id_1': {'v': 2, 'key': [('user_id', 1)]}, 'review_id_1': {'v': 2, 'key': [('review_id', 1)]}}
